In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import seaborn as sns
import math
from IPython.display import Markdown, display
import importlib
import json
from tabulate import tabulate
import colorsys
from scipy.ndimage.filters import gaussian_filter
import sklearn
import matplotlib
import xgboost as xgb
import random
from tqdm import tqdm as tqdm


from torchsummary import summary

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import src.cropnet as cropnet


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def printmd(string):
    display(Markdown(string))
    
import tensorflow as tf
tf.version.VERSION

/tmp/ipykernel_12698/752784679.py:12: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter
2024-12-05 10:20:03.311496: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 10:20:03.336513: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-05 10:20:03.796320: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


'2.16.1'

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
path = "data/full_data/processed/processed_merged.feather"
df = pd.read_feather(path)
f = open(path.replace(".feather", ".json"), "r")
correlation_filters = json.load(f)
f.close()
if 'level_0' in df.columns:
    df = df.drop(columns=['level_0'])
df = df.reset_index()
if 'level_0' in df.columns:
    df = df.drop(columns=['level_0'])

df = df[np.isnan(df['yield']) == False]

In [4]:
target = 'yield_1'
predictors = [c for c in correlation_filters['base'] if c not in ['index', 'x', 'y', 'x_int', 'y_int', 'polygon_id', 'grdkod_mar'] and "yield" not in c and "_pid" not in c]


In [5]:

mX = np.mean(df[predictors], axis=0)
sX = np.std(df[predictors], axis=0)
sX[sX == 0] = 1

mY = np.mean(df[target], axis=0)
sY = np.std(df[target], axis=0)

norm_info = {
    'X': {'mean': mX, 'std': sX},
    'y': {'mean': mY, 'std': sY}
}


In [6]:

dataset = []
for pid in tqdm(set( df['matching_pid'])):
    pdf = df[df['matching_pid'] == pid]
    X = ((pdf.loc[df['matching_pid'] == pid, predictors]-mX)/sX).values
    y = ((pdf.loc[df['matching_pid'] == pid, target]-mY)/sY).values.reshape((-1, 1))
    info = {
        'x': pdf.x.values,
        'y': pdf.y.values,
        'pid': pid
    }
    dataset.append((X, y, info))

100%|██████████| 152/152 [00:06<00:00, 24.13it/s]


In [7]:

x_int = ((info['x'] - np.min(info['x']))/10).astype(int)
np.max(x_int)

81

In [8]:
dataset_CNN = []
for X, y, info in dataset:
    d = 3
    
    x_int = ((info['x'] - np.min(info['x']))/10).astype(int)
    y_int = ((info['y'] - np.min(info['y']))/10).astype(int)
    w = np.max(x_int)+1+2*d
    h = np.max(y_int)+1+2*d

    imgX = np.zeros((w, h, X.shape[1]))
    imgX[x_int+d, y_int+d, :] = X
    imgY = np.zeros((w, h))
    imgY[x_int+d, y_int+d] = y.reshape(-1)

    IMG_BATCH = np.zeros((y.shape[0]//50, X.shape[1], 2*d+1, 2*d+1))
    IMG_Y = np.zeros((y.shape[0]//50, 1))
    xs = np.zeros(y.shape[0]//50)
    ys = np.zeros(y.shape[0]//50)
    for i in range(IMG_BATCH.shape[0]):
        idx = random.randint(0, X.shape[0]-1)
        x = x_int[idx]+d
        y = y_int[idx]+d
        IMG_BATCH[i, :, :, :] = np.transpose(imgX[x-d:x+d+1, y-d:y+d+1, :], (2,0,1))
        IMG_Y[i] = imgY[x, y]
        xs[i] = info['x'][idx]
        ys[i] = info['y'][idx]
        
    IMG_BATCH = torch.tensor(IMG_BATCH, device=device, dtype=torch.float)
    IMG_Y = torch.tensor(IMG_Y, device=device, dtype=torch.float)
    info = {
        'x': xs,
        'y': ys,
        'pid': pid
    }
    dataset_CNN.append((IMG_BATCH, IMG_Y, info))

In [9]:
def split_data(dataset, p_val, p_test, seed=-1):
    
    N = np.sum([X.shape[0] for X, y, info in dataset])
    rem = [data for data in dataset]
    
    data_val = []
    val_size = 0
    while val_size/N < p_val:
        idx = random.choice(range(len(rem)))
        data = rem.pop(idx)
        data_val.append(data)
        val_size += data[0].shape[0]
        
        
    data_test = []
    test_size = 0
    while test_size/N < p_test:
        idx = random.choice(range(len(rem)))
        data = rem.pop(idx)
        data_test.append(data)
        test_size += data[0].shape[0]
        
    return rem, data_val, data_test


In [50]:
M = dataset[0][0].shape[1]

In [107]:
M//4

223

In [108]:
class Conv(torch.nn.Module):
    def __init__(self):
        super(Conv, self).__init__()

        self.pool = nn.MaxPool2d(2,2)
        self.conv_layers = torch.nn.ModuleList([
            nn.Conv2d(M, M//2, 3, padding=1),
            nn.Conv2d(M//2, M//4, 3, padding=1)
        ])
        self.dense_layer = nn.Linear(M//4, 64)
        self.output = nn.Linear(64, 1)

    def forward(self, data):
        x = data
        for layer in self.conv_layers:
            x = layer(x)
            x = self.pool(nn.functional.relu(x))
        x = torch.flatten(x, 1)
        x = self.dense_layer(x)
        x = torch.relu(x)
        x = self.output(x)
        return x
    

model = Conv()
summary(model);

Layer (type:depth-idx)                   Param #
├─MaxPool2d: 1-1                         --
├─ModuleList: 1-2                        --
|    └─Conv2d: 2-1                       3,580,934
|    └─Conv2d: 2-2                       895,345
├─Linear: 1-3                            14,336
├─Linear: 1-4                            65
Total params: 4,490,680
Trainable params: 4,490,680
Non-trainable params: 0


In [109]:


def run_model(model, data_train, data_val, data_test, epochs=100, lr=0.001):
    

    history = {
        'train': [],
        'val': [],
        'test': []
    }


    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()

    scores = {}
    top_score = 10000

    # Training loop
    model.train()

    pbar = tqdm(range(epochs))
    for epoch in pbar:
        
        random.shuffle(data_train)
        err = np.zeros(0)
        total_los = []
        for X, y, info in data_train:
            optimizer.zero_grad()
            pred = model(X)
            y_cpu = y.clone().cpu().detach().numpy()
            pred_cpu = pred.clone().cpu().detach().numpy()
            y_gpu = torch.tensor(y_cpu).to(device)
            loss = criterion(pred, y_gpu)
            loss.backward()
            total_los.append(loss.item())
            optimizer.step()
            err = np.concatenate([err, cropnet.eval_error(pred, y, info, norm_info)])
        train_rmse = np.sqrt(np.nanmean(err**2))
        history['train'].append(train_rmse)

        optimizer.zero_grad()

        val_rmse, val_err = cropnet.eval_dataset(data_val, model, norm_info)
        #val_err = val_err - np.nanmean(val_err)
        history['val'].append(val_rmse)
        val_acc10 = np.mean((np.abs(val_err) < 1)[np.isnan(val_err) == False])
        metric = val_rmse
            
        test_rmse, test_err = cropnet.eval_dataset(data_test, model, norm_info)
        #test_err = test_err - np.nanmean(test_err)
        #test_rmse = np.sqrt(np.nanmean(test_err**2))
        history['test'].append(test_rmse)
        acc5 = np.mean((np.abs(test_err) < 0.5)[np.isnan(test_err) == False])
        acc10 = np.mean((np.abs(test_err) < 1)[np.isnan(test_err) == False])
        acc20 = np.mean((np.abs(test_err) < 2)[np.isnan(test_err) == False])

        if metric < top_score:
            top_score = metric
            torch.save(model, "/".join(["models", "current.pth"]))
            scores = {
                'test_rmse': test_rmse,
                'test_acc_0.5': acc5,
                'test_acc_1.0': acc10, 
                'test_acc_2.0': acc20,
                'test_rel_rmse': test_rmse/norm_info['y']['mean'],
            }

        pbar.set_description(f'Loss: {np.round(np.mean(total_los), 3)}, train rmse: {np.round(train_rmse, 3)}, val rmse: {np.round(val_rmse, 3)}, test rmse: {np.round(test_rmse, 3)}, top: [{np.round(metric, 3)} >= {np.round(top_score, 3)} {[np.round(scores[s], 3) for s in scores]}]')

    model = torch.load("models/current.pth")
    return model

In [111]:
import random
epochs = 100

score_list = []
for i in range(10):
    data_train, data_val, data_test = split_data(dataset_CNN, 0.15, 0.15)
    M = dataset_CNN[0][0].shape[1]
    model = Conv()
    model.to(device)
    model = run_model(model, data_train, data_val, data_test, epochs=100, lr=0.001)

    
    y_tot = np.zeros(0)
    y_pred = np.zeros(0)
    test_err = np.zeros(0)
    test_err_norm = np.zeros(0)
    pids = np.zeros(0)
    for X, y, info in data_test:
        pred = model(X)
        
        yp = pred.cpu().detach().numpy().reshape(-1)
        yt = y.cpu().detach().numpy().reshape(-1)
        err = (yt - yp)*norm_info['y']['std']
        
        y_tot = np.concatenate([y_tot, yt])
        y_pred = np.concatenate([y_pred, yp])
        test_err = np.concatenate([test_err, err])
        test_err_norm = np.concatenate([test_err_norm, err - np.nanmean(err)])
        pids = np.concatenate([pids, np.ones(err.shape)*info['pid']])
        
    y_tot = y_tot*norm_info['y']['std'] + norm_info['y']['mean']
    scores_unnorm = [np.sqrt(np.nanmean(test_err**2)), np.nanmean((np.abs(test_err) < 1)[np.isnan(test_err) == False]), np.nanmean(np.abs(test_err/y_tot)), 1-np.sum(test_err**2)/np.sum((y_tot-np.mean(y_tot))**2)]
    scores_norm = [np.sqrt(np.nanmean(test_err_norm**2)), np.nanmean((np.abs(test_err_norm) < 1)[np.isnan(test_err_norm) == False]), np.nanmean(np.abs(test_err_norm/y_tot)), 1-np.sum(test_err_norm**2)/np.sum((y_tot-np.mean(y_tot))**2)]
    result = [scores_unnorm, scores_norm, [list(y_tot), list(y_pred), list(test_err), list(test_err_norm), list(pids)]]
    score_list.append(result)


cropnet.save_score_to_json(score_list, "results_final/other_models/CNN.json")

  0%|          | 0/100 [00:00<?, ?it/s]

Loss: 0.089, train rmse: 0.845, val rmse: 1.023, test rmse: 1.189, top: [1.023 >= 0.907 [1.025, 0.484, 0.699, 0.939, 0.124]]: 100%|██████████| 100/100 [00:40<00:00,  2.50it/s]
Loss: 0.061, train rmse: 0.68, val rmse: 0.961, test rmse: 1.169, top: [0.961 >= 0.841 [1.313, 0.366, 0.619, 0.898, 0.159]]: 100%|██████████| 100/100 [00:39<00:00,  2.55it/s]
Loss: 0.058, train rmse: 0.638, val rmse: 1.102, test rmse: 1.221, top: [1.102 >= 0.946 [1.497, 0.343, 0.617, 0.879, 0.181]]: 100%|██████████| 100/100 [00:38<00:00,  2.57it/s]
Loss: 0.319, train rmse: 1.509, val rmse: 1.256, test rmse: 1.115, top: [1.256 >= 0.965 [1.05, 0.388, 0.701, 0.935, 0.127]]: 100%|██████████| 100/100 [00:38<00:00,  2.60it/s]
Loss: 0.166, train rmse: 1.047, val rmse: 1.066, test rmse: 1.284, top: [1.066 >= 0.991 [1.193, 0.366, 0.645, 0.894, 0.145]]: 100%|██████████| 100/100 [00:39<00:00,  2.54it/s]
Loss: 0.32, train rmse: 1.373, val rmse: 1.234, test rmse: 1.598, top: [1.234 >= 1.005 [1.141, 0.41, 0.699, 0.908, 0.138]]